In [1]:
from sklearn.datasets import load_sample_images
import tensorflow as tf

images = load_sample_images()["images"]
images = tf.keras.layers.CenterCrop(height=70, width=120)(images)
images = tf.keras.layers.Rescaling(scale=1 / 255)(images)

c:\Users\atthe\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
images.shape

TensorShape([2, 70, 120, 3])

In [3]:
import numpy as np
mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_train_full = np.expand_dims(X_train_full, axis=-1).astype(np.float32) / 255
X_test = np.expand_dims(X_test.astype(np.float32), axis=-1) / 255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

In [4]:
from functools import partial

tf.random.set_seed(42)
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, padding="same",
                        activation="relu", kernel_initializer="he_normal")
model = tf.keras.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28,28,1]),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=256,activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=10, activation="softmax")    
])

c:\Users\atthe\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))
score = model.evaluate(X_test, y_test)
X_new = X_test[:10]
y_pred = model.predict(X_new)

Epoch 1/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 60s 34ms/step - accuracy: 0.7706 - loss: 0.6484 - val_accuracy: 0.8712 - val_loss: 0.3581
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 69s 40ms/step - accuracy: 0.8713 - loss: 0.3727 - val_accuracy: 0.8910 - val_loss: 0.2984
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 67s 39ms/step - accuracy: 0.8893 - loss: 0.3182 - val_accuracy: 0.8990 - val_loss: 0.2811
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 70s 41ms/step - accuracy: 0.9008 - loss: 0.2833 - val_accuracy: 0.8994 - val_loss: 0.2812
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 80s 47ms/step - accuracy: 0.9086 - loss: 0.2583 - val_accuracy: 0.8992 - val_loss: 0.2728
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 85s 49ms/step - accuracy: 0.9147 - loss: 0.2442 - val_accuracy: 0.8938 - val_loss: 0.2911
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 83s 48ms/step - accuracy: 0.9194 - loss: 0.2314 - val_accuracy: 0.8966 - val_loss: 0.2673
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 86s 50ms/step - accuracy: 0.9225 -

In [6]:
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="same", kernel_initializer="he_normal",
                        use_bias=False)

class ResidualUnit(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            tf.keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            tf.keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                tf.keras.layers.BatchNormalization()
            ]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [7]:
model = tf.keras.Sequential([
    DefaultConv2D(64, kernel_size=7, strides=2, input_shape=[224, 224, 3]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"),
])
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters

model.add(tf.keras.layers.GlobalAvgPool2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation="softmax"))

c:\Users\atthe\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model = tf.keras.applications.ResNet50(weights="imagenet")

In [9]:
K = tf.keras.backend
images = K.constant(load_sample_images()["images"])
images_resized = tf.keras.layers.Resizing(height=224, width=224,
                                          crop_to_aspect_ratio=True)(images)

In [10]:
inputs = tf.keras.applications.resnet50.preprocess_input(images_resized)

In [13]:
y_prob = model.predict(inputs)
y_prob.shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


(2, 1000)

In [15]:
top_K = tf.keras.applications.resnet50.decode_predictions(y_prob, top=3)
for image_index in range(len(images)):
    print(f"Image {image_index}")
    for class_id, name, y_prob in top_K[image_index]:
        print(f"{class_id} - {name:12s} {y_prob:.2%}")

35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step
Image 0
n03877845 - palace       54.69%
n03781244 - monastery    24.71%
n02825657 - bell_cote    18.55%
Image 1
n04522168 - vase         32.67%
n11939491 - daisy        17.82%
n03530642 - honeycomb    12.04%


In [17]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
dataset_size = info.splits["train"].num_examples
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\atthe\tensorflow_datasets\tf_flowers\incomplete.SES32F_3.0.1\tf_flowers-train.tfrecord*...:…

Dataset tf_flowers downloaded and prepared to C:\Users\atthe\tensorflow_datasets\tf_flowers\3.0.1. Subsequent calls will reuse this data.


In [18]:
class_names

['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']

In [19]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)

In [22]:
tf.keras.backend.clear_session()

batch_size =32
preprocess = tf.keras.Sequential([
    tf.keras.layers.Resizing(height=224, width=224, crop_to_aspect_ratio=True),
    tf.keras.layers.Lambda(tf.keras.applications.xception.preprocess_input)
])
train_set = train_set_raw.map(lambda X, y: (preprocess(X), y))
train_set = train_set.shuffle(1000, seed=42).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(lambda X, y: (preprocess(X), y)).batch(batch_size)
test_set = test_set_raw.map(lambda X, y: (preprocess(X), y)).batch(batch_size)

In [25]:
base_model = tf.keras.applications.xception.Xception(weights="imagenet",
                                                   include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


In [26]:
for layer in base_model.layers:
    layer.trainable = False

In [27]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=3)

Epoch 1/3
86/86 ━━━━━━━━━━━━━━━━━━━━ 85s 962ms/step - accuracy: 0.8027 - loss: 0.8261 - val_accuracy: 0.8494 - val_loss: 0.7051
Epoch 2/3
86/86 ━━━━━━━━━━━━━━━━━━━━ 100s 1s/step - accuracy: 0.9037 - loss: 0.3817 - val_accuracy: 0.8603 - val_loss: 0.6647
Epoch 3/3
86/86 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accuracy: 0.9440 - loss: 0.1850 - val_accuracy: 0.8512 - val_loss: 0.6839


In [30]:
for layer in base_model.layers[56:]:
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=10)

Epoch 1/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 189s 2s/step - accuracy: 0.8990 - loss: 0.2887 - val_accuracy: 0.8494 - val_loss: 0.6363
Epoch 2/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 205s 2s/step - accuracy: 0.9815 - loss: 0.0648 - val_accuracy: 0.9238 - val_loss: 0.3515
Epoch 3/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 212s 2s/step - accuracy: 0.9964 - loss: 0.0135 - val_accuracy: 0.9074 - val_loss: 0.3330
Epoch 4/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 222s 3s/step - accuracy: 0.9985 - loss: 0.0076 - val_accuracy: 0.9165 - val_loss: 0.2947
Epoch 5/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 221s 3s/step - accuracy: 0.9989 - loss: 0.0043 - val_accuracy: 0.9201 - val_loss: 0.3027
Epoch 6/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 232s 3s/step - accuracy: 0.9964 - loss: 0.0062 - val_accuracy: 0.9129 - val_loss: 0.3150
Epoch 7/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 207s 2s/step - accuracy: 0.9975 - loss: 0.0058 - val_accuracy: 0.9020 - val_loss: 0.3326
Epoch 8/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 197s 2s/step - accuracy: 0.9985 - loss: 0.0049 - val_accuracy: 0.8984 - v